#### Exact product matching over hackathon_dataset.json

* The premisse of this analysis is to match as many products with a very high confidence level in Python using numerical and text methodologies to achive the objective.

In [3]:
import re
import pandas as pd
from pandas.io.json import json_normalize

In [4]:
gzfile = 'dataset.jsonl.gz'
jsonfile = 'hackathon_dataset.json'

In [ ]:
#unzip
import gzip
import shutil
with gzip.open(gzfile, 'rb') as gzf:
    with open(jsonfile, 'wb') as jsonf:
        shutil.copyfileobj(gzf, jsonf)

In [ ]:
df = pd.read_json(jsonfile, lines = True)
print(df.columns)
df.head(10)
#df.to_csv('jsonfile.csv', sep = '|', encoding='utf-8')

Checked the data ouside Python to get the feeling of what the data looks like.

On first notice, there are some fields that can be considered for product matching, such as PRODUCT_ID and NAME
On a closer inspection, each SHOP brings different (scrapped) data and will need some work to normalize.

e.g. 
* Bol doesn't have SPECIFICATIONS nor CONTENTS but similar products have the same product_id. 
* Planit has SPECIFICATIONS that are somewhat easy to parse (not directly but with some regex it's easy to create a viable json structure)
* Praxis has the same SPECIFICATIONS structure as Planit so can apply the same methods and Planit. Doesn't contain BRAND nor CONTENTS.
* Hubo has a different SPECIFICATIONS structure that cannot apply the same method as Planit but can be cleaned and structured. CONTENTS only covers ~21% of the Hubo set. May help a little bit but not much.
* Hornbach appears to have same Praxis structure on SPECIFICATIONS but has some minor character encoding errors that should be easy to fix.

Considering that each shop has data with different characteristics, I decided to divide each shop as its own subset and work on the matching. Once that done for each shop, then I would cross the data between shops.

In [ ]:
df = df.sort_values('DALTIX_ID')

for i in df.SHOP.unique():
    globals()['df_'+str(i)] = df[df['SHOP']==i]

Same process applied on each shop. Due to limited time, was only able to do with BOL.

Bol dataset
* Most of the times when Product_ID is the same, the product is the same but has some exceptions, specially for color. Therefore it sould be taken in account also the name of the product. 

In [ ]:
Dups = df_bol[df_bol[['PRODUCT_ID', 'NAME']].duplicated(keep = False) == True]
duplicates_Bol = pd.DataFrame(columns = ['daltix_id_1','daltix_id_2'])
for i in Dups.index:
    tst = Dups.loc[(Dups[['PRODUCT_ID', 'NAME']]  == Dups.loc[i][['PRODUCT_ID', 'NAME']]) & (Dups.index != i) ]
    sub = pd.DataFrame(columns = ['daltix_id_1','daltix_id_2'])
    sub['daltix_id_2'] =  tst['DALTIX_ID']
    sub['daltix_id_1'] =  Dups['DALTIX_ID'][i]
    duplicates_Bol = pd.concat([duplicates_Bol, sub])
    
#remove mirror pairs
duplicates_Bol = duplicates_Bol.loc[pd.DataFrame(np.sort(duplicates_Bol[['daltix_id_1','daltix_id_2']],1),index=duplicates_Bol.index).drop_duplicates(keep='first').index]

In [ ]:
#The previous code didn't take in account for mirror pairs, only identical pair, so I had to include the following 
#code afterwards to remove mirror pairs

duplicates_Bol = duplicates_Bol.loc[pd.DataFrame(np.sort(duplicates_Bol[['daltix_id_1','daltix_id_2']],1),index=duplicates_Bol.index).drop_duplicates(keep='first').index]

Sometimes there are similar names that are not exact match due to case sensitivity, punctuation or accents. 
For these cases I went looking for normalized names and compare it with the duplicates found previously.

Normalize text: lowercase, strip accents, remove ponctuation

In [ ]:
df['NAME_CLEAN'] = df['NAME'].str.lower()
df['NAME_CLEAN'] = df['NAME_CLEAN'].str.replace('[^\w\s]','')
df['NAME_CLEAN'] = df['NAME_CLEAN'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
Dups = df_bol[df_bol.duplicated(subset = ['NAME_CLEAN'], keep = False) == True]
duplicates_Bol2 = pd.DataFrame(columns = ['daltix_id_1','daltix_id_2'])
for i in Dups.index:
    tst = Dups.loc[(Dups['NAME_CLEAN']  == Dups.loc[i]['NAME_CLEAN']) & (Dups['PRODUCT_ID']  != Dups.loc[i]['PRODUCT_ID']) &(Dups.index != i) ]
    sub = pd.DataFrame(columns = ['daltix_id_1','daltix_id_2'])
    sub['daltix_id_2'] =  tst['DALTIX_ID']
    sub['daltix_id_1'] =  Dups['DALTIX_ID'][i]
    duplicates_Bol2 = pd.concat([duplicates_Bol2, sub])
duplicates_Bol2 = duplicates_Bol2.loc[pd.DataFrame(np.sort(duplicates_Bol2[['daltix_id_1','daltix_id_2']],1),index=duplicates_Bol2.index).drop_duplicates(keep='first').index]

duplicates_Bol3 = pd.concat([duplicates_Bol2, duplicates_Bol]).drop_duplicates(keep = False)
duplicates_Bol_consolidated = duplicates_Bol3.loc[pd.DataFrame(np.sort(duplicates_Bol3[['daltix_id_1','daltix_id_2']],1),index=duplicates_Bol3.index).drop_duplicates(keep='first').index]


Noticed that sometimes there are products that are the same but the word order is not the same. To take that in account decided to try fuzzy matching.

Testing for similarity using fuzzywuzzy
https://github.com/seatgeek/fuzzywuzzy

Used function token_sort_ratio which allows to calculate the sentence similarity (in this case Levenshtein Distance) without considering word order.
Not tested was the function token_set_ratio that ignores duplicate words. 

In [ ]:
NonDups = df_bol[df_bol.duplicated(subset = ['PRODUCT_ID', 'NAME'], keep = False) == False]
similarityMatrix = pd.DataFrame(columns = ['daltix_id_1', 'daltix_id_2', 'similarity'])

from fuzzywuzzy import fuzz
for i in NonDups['DALTIX_ID']:
    for j in Dups['DALTIX_ID']:
        tmp1 = fuzz.token_sort_ratio(NonDups['NAME'][NonDups['DALTIX_ID']==i],NonDups['NAME'][NonDups['DALTIX_ID']==j])       
        similarityMatrix = similarityMatrix.append({'daltix_id_1':i, 'daltix_id_2':j, 'similarity': tmp1 }, ignore_index = True)

This is a very computer intensive process. 
* Another approach is use parallelization in chunks to accelerate the process.
* Tried to test it out but unfortunately didn't have time to debug it properly.

In [ ]:
#import numpy as np
#import concurrent.futures
#import multiprocessing
#num_processes = multiprocessing.cpu_count()
# 
#NonDups = df_bol[df_bol.duplicated(subset = ['PRODUCT_ID', 'NAME'], keep = False) == False]
#SimilarityTable = pd.DataFrame(columns = ['daltix_id_1', 'daltix_id_2', 'similarity'])
# 
#for i in NonDups['DALTIX_ID']:
#    sub = pd.DataFrame(columns = ['daltix_id_1','daltix_id_2'])
#    sub['daltix_id_2'] =  Dups['DALTIX_ID']
#    sub['daltix_id_1'] =  i
#    SimilarityTable = pd.concat([SimilarityTable, sub])

# Define a function on the numbers
#from fuzzywuzzy import fuzz
#def sim(id1, id2):
#    return fuzz.token_sort_ratio(id1,id2)
#
#if __name__ == '__main__':
#    with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
#        SimilarityTable['similarity'] = list(pool.map(sim, SimilarityTable['daltix_id_1'], SimilarityTable['daltix_id_2'], chunksize=1000))

Matching

Bol:
- Matching using ProductId, Name
- Matching using Clean Name

#### Next Steps

Planit
- No same ProductID
- Matching using Name / CleanName (May need to go indepth to check is specs are really the same)

Praxis:
- No same ProductID
- Matching using Name / CleanName (May need to go indepth to check is specs are really the same)

Hornbach:
- Name / CleanName Only (May need to go indepth to check is specs are really the same)

Hubo:
- Name / CleanName Only (May need to go indepth to check is specs are really the same)


* While analysing the data, when crossing inter-shop information it was interesting to find that Planit and Praxis use similar productIDs, however NAMES is not exact match. This is where cleaning and fuzzy matching at high confidence level may come handy.

For a more indepth analysis and matching both SPECIFICATION and CONTENT should be considered. An easy approach was to use regex to extract some key features. 
e.g. 
`re.search("(?<='Breedte':\s)'\"(.*?)\"',", str(df['SPECIFICATIONS']i), re.IGNORECASE)`


### Future work

#### Text / NLP
- Products can be divided into diferent types/categories that have similar features (such as weight, length, capacity, etc.). It would be interestig to separate each product in diferent categories using some multi-class categorization algorithm over the DESCRIPTION. This would facilitate further analysis.
To do this, further enhancement over the normalization of the corpus is needed. Since it's categorization, we won't need to maintain semantical meaning on the corpus, sowe can not only apply lowercase and remove punctuation but also apply stopwords on td-idf basis and stemming or lemmatization (Frog perhaps? https://languagemachines.github.io/frog/)

- Another aproach for products that do not have specs nor contents as a mean to extract structured data, it is to train a NER model to extract from the DESCRIPTION features such as: Brand, size, capacity, color, etc.

e.g.
{
	"text": "Zomerkleed ovaal 180 microns voor een zwembad met een afmeting van 1000 x 550 cm. Het kleed drijft op het water en is dus iets kleiner dan het zwembad zelf. Het draagt bij aan het minder vuil worden van uw zwembadwater wat een gunstige invloed heeft ophet gebruik van zwembadchemie. Tevens houdt de structuur van het noppenfolie de temperatuur van het zwemwater beter vast, omdat verdamping wordt verminderd, terwijl de warmte van de zon wordt doorgegeven. Afmeting: 995 x 545 cm.",

	"tags": [{
			"tag": "product",
			"tag_beggining": 1,
			"tag_end": 10,
			"tag_text": "Zomerkleed"
		},
		{
			"tag": "shape",
			"tag_beggining": 12,
			"tag_end": 17,
			"tag_text": "ovaal"
		},
		{
			"tag": "size",
			"tag_beggining": 468,
			"tag_end": 480,
			"tag_text": "995 x 545 cm"
		}
	]
}


#### Image

- Scrapping the product image from the website may also help when the similarity distance is very high, specially for iter-shop products. 

- Many of the images that appear in the websites use the same product photography. Perhaps using image comparison methodologies may increase the accuracy of the match. Somethin like this OpenCV's Feature Matching and some Homography methodology to take in account rotation/skewness



